In [24]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import numpy as np
import imutils
import time
import cv2
import os

def detect_and_predict_emot(frame, faceNet, maskNet):
	# grab the dimensions of the frame and then construct a blob
	# from it
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (224, 224), (104.0, 177.0, 123.0)) #preprocessing :- image, scaling,size,mean(RGB)  three tupple


    # pass the blob through the network and obtain the face detections
    faceNet.setInput(blob)
    detections = faceNet.forward()
    print(detections.shape)

    # initialize our list of faces, their corresponding locations,
    # and the list of predictions from our face mask network
    faces = [] #list of face
    locs = []
    preds = []

    # loop over the detections
    for i in range(0, detections.shape[2]):
        # extract the confidence (i.e., probability) associated with
        # the detection
        confidence = detections[0, 0, i, 2]

        # filter out weak detections by ensuring the confidence is
        # greater than the minimum confidence
        if confidence > 0.5:
            # compute the (x, y)-coordinates of the bounding box for
            # the object
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            # ensure the bounding boxes fall within the dimensions of
            # the frame
            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(w - 1, endX), min(h - 1, endY))

            # extract the face ROI, convert it from BGR to RGB channel
            # ordering, resize it to 224x224, and preprocess it
            face = frame[startY:endY, startX:endX]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB) 
            face = cv2.resize(face, (224, 224))
            face = img_to_array(face)
            face = preprocess_input(face)

            # add the face and bounding boxes to their respective
            # lists
            faces.append(face)  #more then one face 
            locs.append((startX, startY, endX, endY))

    # only make a predictions if at least one face was detected
    if len(faces) > 0:
        # for faster inference we'll make batch predictions on *all*
        # faces at the same time rather than one-by-one predictions
        # in the above `for` loop
        faces = np.array(faces, dtype="float32")
        preds = emotNet.predict(faces, batch_size=32)

    # return a 2-tuple of the face locations and their corresponding
    # locations
    return (locs, preds)

# load our serialized face detector model from disk
prototxtPath = r"face_detector\deploy.prototxt"
weightsPath = r"face_detector\res10_300x300_ssd_iter_140000.caffemodel"
faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)

# load the face mask detector model from disk
# emotNet = load_model("emotion_detector.model")
emotNet = load_model("model.h5")
# initialize the video stream
print("[INFO] starting video stream...")
vs = VideoStream(src=0).start()
classes=['angry','happy','neutral','sad','surprise']
# loop over the frames from the video stream
while True:
    # grab the frame from the threaded video stream and resize it
    # to have a maximum width of 400 pixels
    frame = vs.read()
    frame = imutils.resize(frame, width=400)

    # detect faces in the frame and determine if they are wearing a
    # face mask or not
    (locs, preds) = detect_and_predict_emot(frame, faceNet, emotNet)

    # loop over the detected face locations and their corresponding
    # locations
    for (box, pred) in zip(locs, preds):
        # unpack the bounding box and predictions
        (startX, startY, endX, endY) = box #locs has 4 variables
#         (angry,happy,neutral,sad,surprise) = pred # [probab, probab]
        predicted_class=classes[np.argmax(pred)]
        print(pred)
        print(predicted_class)


        # determine the class label and color we'll use to draw
        # the bounding box and text
#         label = "Mask" if mask > withoutMask else "No Mask"
        label= predicted_class
        color = (0, 255, 0) 

        # include the probability in the label
        label = "{}: {:.2f}%".format(label, np.max(pred) * 100)

        # display the label and bounding box rectangle on the output
        # frame
        cv2.putText(frame, label, (startX, startY - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)

    # show the output frame
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF

    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break


# do a bit of cleanup
cv2.destroyAllWindows()
vs.stop()


[INFO] starting video stream...
(1, 1, 200, 7)
[8.0188233e-01 1.7010699e-01 4.8094927e-05 1.1178215e-04 2.7850917e-02]
angry
(1, 1, 200, 7)
[5.9224904e-01 1.9516951e-01 2.4206334e-05 8.7724526e-05 2.1246949e-01]
angry
(1, 1, 200, 7)
[6.8895805e-01 1.5384290e-01 1.3356798e-05 7.6480945e-05 1.5710916e-01]
angry
(1, 1, 200, 7)
[6.8382943e-01 8.4759705e-02 6.5462400e-06 8.0127589e-05 2.3132427e-01]
angry
(1, 1, 200, 7)
[3.1902738e-02 6.4510530e-01 6.0449471e-05 7.2210474e-05 3.2285935e-01]
happy
(1, 1, 200, 7)
[9.4612181e-02 4.9828216e-01 5.4297016e-05 1.0151315e-04 4.0694985e-01]
happy
(1, 1, 200, 7)
[4.0073189e-01 1.0840924e-01 1.5756477e-05 9.3711766e-05 4.9074939e-01]
surprise
(1, 1, 200, 7)
[2.7933159e-01 1.5396679e-02 1.5529567e-06 5.1816023e-05 7.0521832e-01]
surprise
(1, 1, 200, 7)
[7.4937081e-01 4.1542074e-04 9.7838438e-08 3.9284088e-05 2.5017446e-01]
angry
(1, 1, 200, 7)
[6.1208200e-01 8.6496334e-04 1.7683202e-07 5.6576810e-05 3.8699627e-01]
angry
(1, 1, 200, 7)
[6.4754802e-01 9.

In [12]:
l=[1.7594839e-03, 9.5492321e-01 ,1.2837408e-05 ,1.0396684e-05, 4.3294121e-02]
o=np.max(l)
print(o)

0.95492321
